# HMM with TensorFlow

In [1]:
import tensorflow as tf
import numpy as np
import sys

## Weather HMM example
Das Wetter bei deinem Übersee-Chatfreund lässt sich durch eine Markowkette $X_1,X_2,\ldots$ mit
Zustandsraum $Q=\{\texttt{sun},\texttt{rain},\texttt{storm}\}$ und Übergangsmatrix
$$A=(A[r,s])_{\scriptsize r,s \in Q} = \begin{pmatrix}
0.7 & 0.2 & 0.1\\
0.3 & 0.5 & 0.2\\
0.2 & 0.6 & 0.2\\
\end{pmatrix}
$$
beschreiben (Reihen und Spalten in der Reihenfolge `sun`, `rain`, `storm`).

Dabei sei $X_i$ das Wetter am $i$-ten Tag und $X_1=\texttt{sun}$. Dein Freund verfolgt vom Wetter abhängig Aktivitäten entweder drinnen 
(`in`) oder draußen (`out`). Sei $\Sigma:=\{\texttt{in},\texttt{out}\}$. Folgende Matrix
beschreibe die vom Wetter abhängenden Wahrscheinlichkeiten (Wkeiten) der Aktivitäten
$$B=\big(B[q,s]\big)_{\scriptsize\begin{array}{l}q\in Q\\s\in \Sigma\end{array}} = \begin{pmatrix}
0.4 & 0.6 \\
0.8 & 0.2 \\
0.9 & 0.1 \\
\end{pmatrix}
.$$ 
Ablesebeispiel: Dein Freund bleibt mit Wkeit 0.9 drinnen, wenn es an dem Tag stürmt (Spalten in der Reihenfolge `in`, `out`).
Beantworte folgende Fragen für das durch $Q,\Sigma,A,B$ und $X_1$ gegebene Hidden-Markow-Modell.
Was ist die Wkeit 
$$P(Y_1=Y_2=Y_3=\texttt{in}),$$

dass dein Freund am allen drei Tagen drinnen bleibt?

![forward DP table](forwardManually.png)

**Solution: P(Y=y) = 0.1308**

## Specify the Model

### Example Model Parameters

In [2]:
n = 3 # number of states
s = 2 # emission alphabet size

In [3]:
A_init = np.array([[7, 2, 1], [3, 5, 2], [2, 6, 2]]) / 10.0
B_init = np.array([[4, 6], [8, 2], [9, 1]]) / 10.0
X1_dist = np.array([1., 0., 0.]) # starts with sun
n, s = B_init.shape # number of states, emission alphabet size
y = np.array([0, 0, 0]) # in, in, in

In [4]:
print("transitions:\n", A_init, "\nemissions:\n", B_init)

transitions:
 [[0.7 0.2 0.1]
 [0.3 0.5 0.2]
 [0.2 0.6 0.2]] 
emissions:
 [[0.4 0.6]
 [0.8 0.2]
 [0.9 0.1]]


#### Forward recursion
$$\alpha[i,q] = B[q, y[i]] \sum_{q'} \alpha[i-1, q'] \cdot A[q',q] $$

In [5]:
# tf variants of the transition and emission matrix
A = tf.Variable(A_init, trainable = True)
B = tf.Variable(B_init, trainable = True)

### Forward Variables and Algorithm
$$ \alpha(q,i) = \sum_{x_1,\ldots, x_{i-1}\in Q} P(x_1,\ldots, x_{i-1}, X_i=q, y_1,\ldots, y_i)$$
Initialization: 
$$ \alpha(q, 1) = \sum_{q\in Q} P(X_1 = q)\cdot B[q,y[0]]$$

In [6]:
def forward(y # observation sequence
           ):
    """ Forward Algorithm for Computing Sequence Likelihood """
    ell = y.shape[0]
    α = tf.Variable(np.zeros([ell, n]), trainable = False)
    
    # initialization
    α[0].assign(tf.multiply(B[:, y[0]], X1_dist))
    
    # forward algorithm
    for i in range(1, ell):
        # compute i-th row of DP table
        R = tf.linalg.matvec(A, α[i-1], transpose_a = True)
        α[i].assign(tf.multiply(B[:, y[i]], R))
    return α

def emiProb(α):
    return np.sum(α[-1,:])

In [7]:
α = forward(y)
α.numpy()

array([[0.4    , 0.     , 0.     ],
       [0.112  , 0.064  , 0.036  ],
       [0.04192, 0.0608 , 0.02808]])

In [8]:
Py = emiProb(α)
Py

0.1308

## A HMM as a Special Case of a Recurrent Neural Network
We use the notation of RNNs similar to that in [Dive into Deep Learning](https://d2l.ai/chapter_recurrent-neural-networks/bptt.html). $h_t$ is a size $n$ vector of RNN-"hidden states" (these are real numbers, not to be confused with the hidden states of HMMs, which are from $Q$).  
$$ h_t = f(x_t, h_{t-1}; A, B)$$
We chose the outputs
$$ o_t = \text{sum}(h_t) = h_t[0] + \cdots + h_t[n-1] \in [0,1]$$
so that the final output $o_T$ is just the likelihood of the sequence $P(Y)$.
This RNN does not need to produce intermediate outputs $o_t$ for $t<T$ as they are not used yet. However, they could be used in conjunction with a backwards pass.

### HMMCell
As a template we use the code for [tf.keras.layers.SimpleRNNCell](https://github.com/tensorflow/tensorflow/blob/v2.4.1/tensorflow/python/keras/layers/recurrent.py#L1222-L1420)

In [9]:
from HMMCell import HMMCell

## Test the HMM cell

In [10]:
print (f"n={n}, s={s}")

A_init = np.array([[7, 2, 1], [3, 5, 2], [2, 6, 2]]) / 10.0
B_init = np.array([[4, 6], [8, 2], [9, 1]]) / 10.0
I_init = np.array([1, 1e-10, 1e-10]) # start with X1=sun (very likely)
A_init = np.log(A_init)
B_init = np.log(B_init)
I_init = np.log(I_init)


A_initializer = tf.keras.initializers.Constant(A_init)
B_initializer = tf.keras.initializers.Constant(B_init)
I_initializer = tf.keras.initializers.Constant(I_init)

yi = np.array([[1., 0]]).astype(np.float32) # np.random.random([batch_size, s]).astype(np.float32)
states = np.array([[.4, 0, 0]]).astype(np.float32) # np.random.random([batch_size, n]).astype(np.float32)
hmmC = HMMCell(n,
               transition_initializer=A_initializer,
               emission_initializer=B_initializer,
               init_initializer=I_initializer
              )

#output = hmmC(yi, [0, states, 0.])
#print("output:\n", output[0])

n=3, s=2


In [13]:
inputs = np.array([[[1, 0],[1, 0],[1, 0]]]).astype(np.float32)
hmm = tf.keras.layers.RNN(
      HMMCell(n, transition_initializer=A_initializer,
              emission_initializer=B_initializer,
              init_initializer=I_initializer),
      return_sequences = True,
      return_state = True)
  
alpha, _, lastcol, loglik = hmm(inputs)
alpha = alpha[1]

print()
with np.printoptions(precision=5, suppress=True):
    print ("α=\n", alpha.numpy(),
       "\nlast column of forward table:", lastcol.numpy(),
      "\nlikelihood=", loglik.numpy())


α=
 [[[0.4     0.      0.     ]
  [0.112   0.064   0.036  ]
  [0.04192 0.0608  0.02808]]] 
last column of forward table: [[0.04192 0.0608  0.02808]] 
likelihood= [0.1308]
